In [6]:
from deeplift.visualization import viz_sequence
import shap
import numpy as np
np.random.seed(1)
import random
import os
import tensorflow as tf

In [ ]:
model  = tf.keras.models.load_model('../experiments/2021-08-22/conv_atac_from_config/models/ConvATACseqModel/')

2021-08-22 09:00:50.310233: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcuda.so.1
2021-08-22 09:00:50.346606: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1733] Found device 0 with properties: 
pciBusID: 0000:81:00.0 name: Tesla P100-PCIE-16GB computeCapability: 6.0
coreClock: 1.3285GHz coreCount: 56 deviceMemorySize: 15.90GiB deviceMemoryBandwidth: 681.88GiB/s
2021-08-22 09:00:50.347007: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /hpcf/authorized_apps/rhel7_apps/R/install/4.0.2/lib64/R/lib:/hpcf/authorized_apps/rhel7_apps/geos/install/3.8.1/lib:/hpcf/authorized_apps/rhel7_apps/gdal/install/3.1.2/lib:/hpcf/authorized_apps/rhel7_apps/proj/install/v6/lib:/hpcf/authorized_apps/rhel7_apps/gcc/install/8.3.0/lib64:/hpcf/lsf/lsf_prod/10.1/linux3.10-glibc2

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
seqs_to_explain = onehot_data[[0,3,9]] #these three are positive for task 0
#Compute contributions w.r.t. the linear layer before the very final nonlinearity,
# as recommended in the DeepLIFT paper, to avoid saturation effects
dinuc_shuff_explainer = shap.DeepExplainer((keras_model.input, keras_model.layers[-2].output[:,0]), shuffle_several_times)
dinuc_shuff_explanations = np.sum(dinuc_shuff_explainer.shap_values(seqs_to_explain),axis=-1)[:,:,None]*seqs_to_explain

#compare DeepExplainer calls that rely on a background generation function to those that use the standard API
indiv_bg_explanations = []
for to_explain in seqs_to_explain:
        to_explain_seq = "".join([pos_to_seq[x] for x in np.argmax(to_explain, axis=-1)])
        shuffled_sequences = [dinuc_shuffle(to_explain_seq) for i in range(100)]
        background = np.array([one_hot_encode_along_channel_axis(x) for x in shuffled_sequences])
        shap_values = shap.DeepExplainer((keras_model.input, keras_model.layers[-2].output[:,0]), background).shap_values(to_explain[None,:,:])
        indiv_bg_explanations.append((np.sum(shap_values,axis=-1)[:,:,None]*to_explain[None,:,:]).squeeze())   

for indiv_bg_explanation, dinuc_shuff_explanation in zip(indiv_bg_explanations, dinuc_shuff_explanations):
    print("With standard API")
    viz_sequence.plot_weights(indiv_bg_explanation, subticks_frequency=20)
    print("With dynamic reference")
    viz_sequence.plot_weights(dinuc_shuff_explanation, subticks_frequency=20)